In [1]:
import torch
import numpy as np

In [2]:
NORM = ord('a')-1
OUTPUT_STRING = "The strategy correctly guesses {:.2f}% of the dictionary words"

In [3]:
with open('../data/words_alpha.txt', 'r') as f: 
    data = f.read() 
words = data.splitlines()

In [4]:
letter_map = {chr(i): i-97 for i in range(97,123)}
position_map = {i-97: chr(i) for i in range(97,123)}

In [5]:
transition_probabilities = np.zeros((26,26))

In [6]:
transition_probabilities = np.zeros((26,26))
prior_probabilities = np.zeros((26,))
for word in words:
    for i in range(len(word)-1):
        transition_probabilities[letter_map[word[i]]][letter_map[word[i+1]]] += 1
        prior_probabilities[letter_map[word[i]]] += 1
    prior_probabilities[letter_map[word[-1]]] += 1
prior_probabilities /= prior_probabilities.sum()
transition_probabilities /= transition_probabilities.sum(axis=1).reshape((26,1))
prior_probabilities, transition_probabilities

(array([0.08464029, 0.01829659, 0.04377464, 0.03238966, 0.10772156,
        0.01122788, 0.02364355, 0.02643092, 0.08956661, 0.00156122,
        0.00767277, 0.05577454, 0.03010506, 0.07194762, 0.07199369,
        0.03252444, 0.00168341, 0.07043304, 0.07161769, 0.06607014,
        0.03762703, 0.00946435, 0.00641172, 0.00300255, 0.02019633,
        0.00422269]),
 array([[9.12607947e-04, 4.73768785e-02, 6.44981193e-02, 3.43748993e-02,
         2.19419581e-02, 7.68380103e-03, 3.05813134e-02, 5.18576045e-03,
         2.35774947e-02, 1.13807579e-03, 1.09262434e-02, 1.45154768e-01,
         4.09886228e-02, 1.43308079e-01, 1.39217448e-03, 3.77998633e-02,
         1.22038945e-03, 1.13639373e-01, 5.95414056e-02, 1.50512313e-01,
         1.95298101e-02, 1.25080972e-02, 6.98592436e-03, 4.60240714e-03,
         9.34438961e-03, 5.27523182e-03],
        [1.37197289e-01, 2.70287348e-02, 6.16084546e-03, 7.14026191e-03,
         1.39945974e-01, 2.27477371e-03, 1.37434245e-03, 2.95404641e-03,
         1.4

In [7]:
prior_order = np.argsort(prior_probabilities)

In [8]:
def game_mc(word, is_print=True):
    if is_print: print(f'word: {word}')
    guess_position = 1
    correct_letters_needed = len(set(word))
    transition_order = prior_order
    guess = position_map[transition_order[-guess_position]]
    correct_letters = set()
    n_mistakes = 0
    n_correct = 0
    while n_mistakes < 8 and n_correct < correct_letters_needed:
        if guess in word:
            n_correct += 1
            if is_print: print(f'correct guess: {guess}')
            correct_letters.update(guess)
            guess_position = 1
            transition_probs = transition_probabilities[letter_map[guess]]
            transition_order = np.argsort(transition_probs)
            guess = position_map[transition_order[-guess_position]]
            while guess in correct_letters:
                guess_position += 1
                guess = position_map[transition_order[-guess_position]]
        else:
            if is_print: print(f'wrong guess: {guess}')
            n_mistakes += 1
            guess_position += 1
            guess = position_map[transition_order[-guess_position]]
            while guess in correct_letters:
                guess_position += 1
                guess = position_map[transition_order[-guess_position]]
    if (n_correct == correct_letters_needed):
        if is_print: print('WIN!')
    return n_correct == correct_letters_needed

In [9]:
np.random.seed(seed=42)
results = []
# rpw = torch.randperm(len(words)).tolist() 
# n_tests = 1000
# sample_words = [words[i] for i in rpw[:n_tests]]
is_print = False
for word in words:
    if is_print: print(f'GAME: {i}')
    if is_print: print('-'*20)
    results.append(game_mc(word, is_print=is_print))
    if is_print: print('-'*20)
print(OUTPUT_STRING.format((sum(results) / len(words)*100)))

The strategy correctly guesses 7.21% of the dictionary words


In [10]:
MAX_N_LETTERS = 31
tp = [np.zeros((26,26)) for i in range(MAX_N_LETTERS)]
pp = [np.zeros((26,)) for i in range(MAX_N_LETTERS)]
for word in words:
    n = len(word)-1
    for i in range(n):
        tp[n][letter_map[word[i]]][letter_map[word[i+1]]] += 1
        pp[n][letter_map[word[i]]] += 1
    pp[n][letter_map[word[-1]]] += 1
for i in range(MAX_N_LETTERS):
    pp[i] /= pp[i].sum()
    tp[i] /= tp[i].sum(axis=1).reshape((26,1))

C:\Users\Ilya\AppData\Local\Temp\ipykernel_1688\896290127.py:12: RuntimeWarning: invalid value encountered in divide
  tp[i] /= tp[i].sum(axis=1).reshape((26,1))
C:\Users\Ilya\AppData\Local\Temp\ipykernel_1688\896290127.py:11: RuntimeWarning: invalid value encountered in divide
  pp[i] /= pp[i].sum()


In [11]:
def game_mc_ld(word, is_print=True):
    if is_print: print(f'word: {word}')
    WORD_LEN = len(word)-1
    guess_position = 1
    correct_letters_needed = len(set(word))
    transition_order = np.argsort(pp[WORD_LEN])
    guess = position_map[transition_order[-guess_position]]
    correct_letters = set()
    n_mistakes = 0
    n_correct = 0
    while n_mistakes < 8 and n_correct < correct_letters_needed:
        if guess in word:
            n_correct += 1
            if is_print: print(f'correct guess: {guess}')
            correct_letters.update(guess)
            guess_position = 1
            transition_probs = tp[WORD_LEN][letter_map[guess]]
            transition_order = np.argsort(transition_probs)
            guess = position_map[transition_order[-guess_position]]
            while guess in correct_letters:
                guess_position += 1
                guess = position_map[transition_order[-guess_position]]
        else:
            if is_print: print(f'wrong guess: {guess}')
            n_mistakes += 1
            guess_position += 1
            guess = position_map[transition_order[-guess_position]]
            while guess in correct_letters:
                guess_position += 1
                guess = position_map[transition_order[-guess_position]]
    if (n_correct == correct_letters_needed):
        if is_print: print('WIN!')
    return n_correct == correct_letters_needed

In [12]:
results = []
is_print = False
for i, word in enumerate(words):
    if is_print: print(f'GAME: {i}')
    if is_print: print('-'*20)
    results.append(game_mc_ld(word, is_print=is_print))
    if is_print: print('-'*20)
print(OUTPUT_STRING.format((sum(results) / len(words)*100)))

The strategy correctly guesses 8.19% of the dictionary words
